In [67]:
from tkinter import W
import cv2
import numpy as np
import matplotlib.pyplot as plt
import scipy
import skimage as sk
import vid2data as v
import os
import pytesseract

WHEIGHT = v.WHEIGHT
WHEIGHT2 = v.WHEIGHT2
WWIDTH = v.WWIDTH
WRIGHT = v.WRIGHT
WTOP = v.WTOP

def process_image(image, threshold=0.7, value_on=1, value_off=0):
    """
    Reads an image, converts it to grayscale, and threshold it to create a binary image.

    Parameters:
        image (numpy.ndarray): Input image.
        threshold (float): Threshold value for binary conversion.
        value_on (float): Value assigned to pixels above the threshold.
        value_off (float): Value assigned to pixels below or equal to the threshold.

    Returns:
        numpy.ndarray: Binary image.
    """
    image = image.astype(np.float32) / 255
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image[image > threshold] = value_on
    image[image <= threshold] = value_off
    return image

def visualize_convolution(image, letter):
    """
    Visualizes the convolution of an image with a letter.

    Parameters:
        image (numpy.ndarray): Input image.
        letter (numpy.ndarray): Letter to be convolved.
    """

    # Convolve the image with the letter
    result = scipy.signal.correlate2d(image, letter, mode='same')
    result[result < 0] = 0
    result = result / np.max(result)
    result = result**5  # To make the peaks more pronounced
    max_brightness = np.max(result, axis=0)

    # Plot the image
    plt.figure(figsize=(16, 5))
    plt.imshow(image, cmap='gray')
    plt.axis('off')
    plt.show()

    # Plot the convolution
    plt.figure(figsize=(16, 16))
    plt.imshow(result, cmap='gray')
    plt.axis('off')
    plt.show()

    # Plot the maximum brightness along the horizontal axis
    plt.figure(figsize=(16, 3))
    plt.plot(max_brightness)
    plt.xlim(0, len(max_brightness))
    plt.axis('off')
    plt.show()

def get_letters_from_image(image, threshold=0.7, value_on=255, value_off=0, binary=False, show=False, full_image=False):
    """
    Isolates the letters in an image.

    Parameters:
        path (str): Path to the input image.
        threshold (float): Threshold value for binary conversion.
        value_on (float): Value assigned to pixels above the threshold.
        value_off (float): Value assigned to pixels below or equal to the threshold.
        show (bool): Whether to display the image with bounding boxes.
        binary (bool): whether to use a binary image for similarity
        full_image (bool): whether the image is just the code or the whole screenshot

    Returns:
        list: List of isolated letters.
    """
    #image = cv2.imread(path)
    if full_image:
        #image = v.to_code(image)
        image = image[:, v.REMOVE:]
        image = image[v.HAZTOP:v.HAZBOTTOM, v.HAZLEFT+178:v.HAZRIGHT+200]
    binary_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _,binary_image = cv2.threshold(binary_image, 127, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    contours,hierarchy = cv2.findContours(binary_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cv2.imwrite('temp/a.png', cv2.drawContours(binary_image, contours, -1, (0, 255, 0), 3))
    #binary_image = v.image_prettify(image) # 0 because we want to keep the background for cv2.connectedComponentsWithStats
    _, _, stats, centroids = cv2.connectedComponentsWithStats(binary_image.astype(np.uint8))
    if show:
        #binary_image_with_boxes = process_image(image, threshold, value_on, value_off)
        image_with_boxes = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(16, 5))
        plt.imshow(binary_image, cmap='gray')
        for x, y, w, h, _ in stats[1:]:
            plt.plot([x, x+w, x+w, x, x], [y, y, y+h, y+h, y])
        #for centroid in centroids:
        #    plt.plot(centroid[0], centroid[1], 'bo')
        plt.axis('off')
        plt.show()
    image = process_image(image, threshold, 255, 0) if binary else image
    letters = []

    for i in range(1, len(stats)):
        if stats[i][0] > 170 and stats[i][0] < 235:
            letters.append(isolate_letter(image, stats[i], binary))
    letters = [x for _, x in sorted(zip(centroids[1:, 0], letters))]

    return letters

def isolate_letter(image, stats, binary):
    """
    Isolates a letter from the list of letters.

    Parameters:
        letters (list): List of letters.
        index (int): Index of the letter to be isolated.
        pad (int): Padding value for the isolated letter.
        value (float): Value for padding.

    Returns:
        numpy.ndarray: Isolated letter.
    """
    x, y, w, h, _ = stats
    # (34, 22) these values obtained from looking at maximum size while processing SELKcode.png using pad values of 5
    # (24, 21) these values obtained from looking at 1wave.JPG maximum size, pad value 5
    pad_x = (21 - w) // 2
    pad_y = (24 - h) // 2
    pad_x1 = pad_x + 1 if (21 - w) % 2 == 1 else pad_x
    pad_y1 = pad_y + 1 if (24 - h) % 2 == 1 else pad_y
    if binary:
        return np.pad(image[y:y+h, x:x+w], ((pad_y, pad_y1), (pad_x1, pad_x)), mode='constant', constant_values=0)
    else:
        return np.pad(image[y:y+h, x:x+w], ((pad_y, pad_y1), (pad_x1, pad_x), (0, 0)), mode='constant', constant_values=0)

def similarity(image, letter, true_letter, show=False):
    """
    Convolves an image with a letter and returns the maximum brightness.

    Parameters:
        image (numpy.ndarray): Input image.
        letter (numpy.ndarray): Letter to be convolved.

    Returns:
        float: Maximum brightness after convolution.
    """
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    letter = cv2.cvtColor(letter, cv2.COLOR_BGR2GRAY)
    result = scipy.signal.correlate2d(image, letter, mode='same')
    result[result < 0] = 0
    max_brightness = np.max(result)
    return max_brightness

def predict_letter_from_image(img_letter, chars, binary=False, debug=False, show=False):
    """
    Predicts the letter in an image.

    Parameters:
        img_letter (numpy.ndarray): Image containing a single letter.
        chars (str): String containing the set of characters to predict.
        threshold (float): Threshold value for binary conversion.
        value_on (float): Value assigned to pixels above the threshold.
        value_off (float): Value assigned to pixels below or equal to the threshold.

    Returns:
        str: Predicted letter.
    """
    results = []
    ext = 'proc/' if binary else 'raw/'
    for char in chars:
        letter = cv2.imread('letter_data/' + ext + char + '.png')
        if binary:
            letter = cv2.cvtColor(letter, cv2.COLOR_BGR2GRAY)
        results.append(similarity(img_letter, letter, binary, show=show))
    return chars[np.argmax(results)]

def predict_word_from_letters(letters, chars, show=False, binary=False):
    """
    Predicts the word from a list of letters.

    Parameters:
        letters (list): List of letters.
        chars (str): String containing the set of characters to predict.
        threshold (float): Threshold value for binary conversion.
        pad (int): Padding value for isolated letters.
        value_on_letter (float): Value assigned to pixels above the threshold for letter convolution.
        value_off_letter (float): Value assigned to pixels below or equal to the threshold for letter convolution.

    Returns:
        str: Predicted word.
    """
    results = []
    for i in range(len(letters)):
        if i == 4 or i == 8 or i == 12:
            results.append('-')
        predicted = predict_letter_from_image(letters[i], chars, show=show, binary=binary)
        results.append(predicted)
    return ''.join(results)

def getting_data(path):
    img = cv2.imread(path)
    img = v.to_code(img)
    processed_img = process_image(img, 0.7, 255, 0)
    _, _, stats, _ = cv2.connectedComponentsWithStats(processed_img.astype(np.uint8))
    if len(stats) != 20:
        print('Number of detected letters is not 19. Please try again')
        exit(0)
    #assume format is 'testing-images/1wave.JPG'
    path = path.split('/')[1].split('.')[0]
    # len(stats) - 3 so that hyphens are removed
    for i in range(1, len(stats) - 3):
        letter = isolate_letter(img, stats[i], False)
        processed_letter = isolate_letter(img, stats[i], True)
        cv2.imwrite('temp/1_' + path + str(i) + '.png', letter)
        cv2.imwrite('temp/1_' + path + str(i) + '_proc.png', processed_letter)

def max_value(image):
    h, w, _ = image.shape
    max = 0
    for x in range(w):
        for y in range(h):
            value = np.sum(image[y, x])
            if  value > max:
                max = value
    return max

def getweap(frame, waves):
    info = frame[:, v.REMOVE:]
    right = WRIGHT
    top = WTOP
    weaps = []
    for i in range(0, waves):
        weap1 = info[top:top+WHEIGHT, right-WWIDTH:right]
        top = top + WHEIGHT + WHEIGHT2
        weap2 = info[top:top+WHEIGHT, right-WWIDTH:right]
        top = top + WHEIGHT + WHEIGHT2
        weap3 = info[top:top+WHEIGHT, right-WWIDTH:right]
        top = top + WHEIGHT + WHEIGHT2
        weap4 = info[top:top+WHEIGHT, right-WWIDTH:right]
        right = right - WWIDTH
        top = WTOP

        max = [[0, ''], [0, ''], [0, ''], [0, '']]
        for filen in os.listdir('./weapon-images'):
            if filen.endswith('.png'):
                weap = cv2.imread('./weapon-images/' + filen)
                # print shape of both
                similarity = (v.ssim(weap1, weap), v.ssim(weap2, weap), v.ssim(weap3, weap), v.ssim(weap4, weap))
                #similarity = (histogram(weap1, weap), histogram(weap2, weap), histogram(weap3, weap), histogram(weap4, weap))
                for j in range(0, 4):
                    if similarity[j] > max[j][0]:
                        max[j][0] = similarity[j]
                        if '_' in filen:
                            # file format is _# attached to the end of the name of the weapon, so remove that
                            max[j][1] = filen[:-6]
                        else:
                            max[j][1] = filen[:-4]
        # Rollers are hard to predict, so without having every weapon always check for multiple images of rollers, only check if predicted weapon is one of the rollers
        for j in range(4):
            match j:
                case 0:
                    weapx = weap1
                case 1:
                    weapx = weap2
                case 2:
                    weapx = weap3
                case 3:
                    weapx = weap4
            rollers = ['Roller', 'Carbon', 'Dynamo', 'Flingza', 'Swig']
            if max[j][1] in rollers:
                for filen in os.listdir('./weapon-images/Rollers'):
                    if filen.endswith('.png'):
                        weap = cv2.imread('./weapon-images/Rollers/' + filen)
                        similarity = v.ssim(weapx, weap)
                        if similarity > max[j][0]:
                            max[j][0] = similarity
                            if '_' in filen:
                                max[j][1] = filen[:-6]
                            else:
                                max[j][1] = filen[:-4]
        cv2.imwrite('temp/weap1' + str(i) + max[0][1] + '.png', weap1)
        cv2.imwrite('temp/weap2' + str(i) + max[1][1] +'.png', weap2)
        cv2.imwrite('temp/weap3' + str(i) + max[2][1] +'.png', weap3)
        cv2.imwrite('temp/weap4' + str(i) + max[3][1] +'.png', weap4)
        weaps.append(max[0][1] + ', ' + max[1][1] + ', ' + max[2][1] + ', ' + max[3][1])
    return weaps

def getdate(frame):
    """Gets date from image."""
    frame = frame[:, v.REMOVE:]
    date = frame[v.DATETOP:v.DATEBOTTOM, v.DATELEFT:v.DATERIGHT]
    # remove the background so it doesn't interfere
    # counts the columns that have majority black pixels, and the image can be cut off at the end where there are many columns with black pixels in a row
    h, w, _ = date.shape
    stop_idx = w
    stop = 0
    for x in range(w):
        pixels = 0
        for y in range(h):
            val = np.sum(date[y, x])
            # sum of 100 is considered a black pixel
            if val < 100:
                pixels += 1
        if pixels > 15:
            stop += 1
        else:
            stop = 0
        if stop > 7:
            stop_idx = x
    date = date[:, :stop_idx]
    # assume last two letters are PM or AM, separate the two parts and use detect_text with different config
    gray = cv2.cvtColor(date, cv2.COLOR_BGR2GRAY)
    # unsure if binary or otsu is better
    _, binary = cv2.threshold(gray, 178, 255, cv2.THRESH_BINARY)
    _, _, stats, centroids = cv2.connectedComponentsWithStats(binary)
    indices = np.argsort(centroids[1:, 0])
    # the colon has two separate components, remove both as well as the 'M'
    indices = np.delete(indices, [-5, -6, -1])
    stats = stats[1:]
    letters = []
    for i in range(len(indices)):
        # (18, 12) these values obtained from looking at 1 47 2083PM.png maximum letter size, pad value 3
        ind = indices[i]
        if i == len(indices) - 2:
            stats[ind][2] += 1
        letters.append(v.isolate_letter(date, stats[ind], [18, 12]))

    # go through each letter and compare each letter to the ground truth data and choose the one that is most similar
    date = ''
    letter_list = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'slash']
    slash_ind = 0 # will be used to find where the space should be inserted into the date
    for i in range(len(letters) - 1):
        results = []
        for letter in letter_list:
            letter_base = cv2.imread('letter_data/' + letter + '.png')
            result = v.ssim(letters[i], letter_base)
            results.append(result)
        result = letter_list[np.argmax(results)]
        # the filename can't contain a slash, so this is the workaround
        if result == 'slash':
            result = '/'
            slash_ind = i
        date += result
        if i == len(letters) - 4:
            date += ':'
        cv2.imwrite(str(i) + '.png', letters[i])
    result = np.sum(letters[-1][:, 6:])
    if result < 23298: #
        date += 'PM'
    else:
        date += 'AM'
    # the space is after the 4 digits of the year after the second slash
    date = date[:slash_ind+5] + ' ' + date[slash_ind+5:]
    return date

def get_wave_type(wave, event_key, water_key, lang='en'):
    """
    Gets the wave type based on levenshtein distance.

    Parameters:
        wave: image for text detection
        event_key: key containing night wave names
        water_key: key containing tide level names
        lang: language
    Returns:
        string containing tide level and event if applicable
    """
    wave = np.pad(wave, ((5, 5), (15, 5), (0, 0)), mode='constant', constant_values=0)
    tess = v.detect_text(wave, config='--psm 6')
    print('Raw detected: ', tess)
    if '\n' in tess:
        tess = tess.split('\n')
        water = v.leven(tess[0], water_key)
        event = v.leven(tess[1], event_key)
        return water + '\n' + event
    else:
        return v.leven(tess, water_key)
def getwavetypes(image, waves):
    info = image[:, v.REMOVE:]
    top = v.WAVETYPETOP[waves - 1]
    left = v.WAVETYPELEFT
    WAVEHEIGHT2 = v.WAVEHEIGHT2
    WAVEHEIGHT = v.WAVEHEIGHT
    WAVEWIDTH = v.WAVEWIDTH
    event_key = v.read_keys('keys/event.csv')
    water_key = v.read_keys('keys/water.csv')
    wave1 = get_wave_type(info[top:top+WAVEHEIGHT, left:left+WAVEWIDTH], event_key, water_key)
    wave2 = '-'
    wave3 = '-'
    wave4 = '-'
    if waves > 1:
        top = top + WAVEHEIGHT2
        wave2 = get_wave_type(info[top:top+WAVEHEIGHT, left:left+WAVEWIDTH], event_key, water_key)
    if waves > 2:
        top = top + WAVEHEIGHT2
        wave3 = get_wave_type(info[top:top+WAVEHEIGHT, left:left+WAVEWIDTH], event_key, water_key)
        cv2.imwrite('temp/testing.png', info[top:top+WAVEHEIGHT, left:left+WAVEWIDTH])
    if waves > 3:
        top = top + WAVEHEIGHT2
        wave4 = get_wave_type(info[top:top+WAVEHEIGHT, left:left+WAVEWIDTH], event_key, water_key)
    return wave1, wave2, wave3, wave4

def histogram(image1, image2):
    hist_img1 = cv2.calcHist([image1], [0, 1, 2], None, [256, 256, 256], [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist_img1, hist_img1, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
    hist_img2 = cv2.calcHist([image2], [0, 1, 2], None, [256, 256, 256], [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist_img2, hist_img2, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
    # Find the metric value
    return cv2.compareHist(hist_img1, hist_img2, cv2.HISTCMP_CORREL)

def gethazard(frame, lang='en'):
    """
    Gets hazard from image.
    lang is needed because the position of the hazard differs depending on language.
    Uses hazard_data to compare images using ssim
    """
    # method using pytesseract instead
    #hazard = detect_text(info[HAZTOP:HAZBOTTOM, HAZLEFT:HAZRIGHT], '--psm 7, -c tessedit_char_whitelist=0123456789')
    info = frame[:, v.REMOVE:]
    threshold = 0.7 # determined through testing
    haz = info[v.HAZTOP:v.HAZBOTTOM, v.HAZLEFT+v.HAZLANG[lang]:v.HAZRIGHT]
    # binarize image for cv2.connectedComponentsWithStats
    #binary = process_image(haz)
    gray = cv2.cvtColor(haz, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 178, 255, cv2.THRESH_BINARY)
    _, _, stats, centroids = cv2.connectedComponentsWithStats(binary)
    # remove components that are too short, which are sometimes erroneously picked up
    length = len(stats)
    for i in range(length):
        if i >= length:
            break
        if stats[i][3] < 10:
            stats = np.delete(stats, i, axis=0)
            centroids = np.delete(centroids, i, axis=0)
            length = length - 1

    # remove the percent symbol, which is at the max x value
    max = np.argmax(centroids[:, 0])
    stats = np.delete(stats, max, axis=0)
    centroids = np.delete(centroids, max, axis=0)

    # separate each digit into its own image
    digits = []
    for i in range(1, len(stats)):
        # (24, 21) these values obtained from looking at 1wave.JPG maximum digit size, pad value 5
        digits.append(v.isolate_letter(haz, stats[i], [24, 21]))
    digits = [x for _, x in sorted(zip(centroids[1:, 0], digits))]
    hazard = ''
    digit_list = '0123456789'
    #cv2.imwrite('temp/temp.png', digits[3])

    # go through each digit and compare each digit to the ground truth data and choose the one that is most similar
    for digit_img in digits:
        results = []
        for digit in digit_list:
            digit_base = cv2.imread('hazard_data/' + digit + '.png')
            results.append(v.ssim(digit_img, digit_base))
        hazard += digit_list[np.argmax(results)]
    return hazard

#def click_event(event, x, y, flags, param):
#    if event == cv2.EVENT_LBUTTONDOWN:
#        print("Coordinates (x, y):", x, ",", y)
#        pixel_value = image[y, x]  # OpenCV uses (y, x) for accessing pixel values
#        print("Pixel value (BGR):", pixel_value)
# Constants
#CHARS = "ABCDEFGHJKLMNPQRSTUVWXY0123456789"
CHARS = "0123456789" #for hazard
threshold_binary = 0.7
value_on_binary, value_off_binary = 255, 0
value_on_letter_convolution, value_off_letter_convolution = 5, -1
pad = 5
#path = 'temp/temp.png'
#image = cv2.imread(path)
cam = cv2.VideoCapture('videos/codes14.mp4')
for i in range(147):
    cam.read()
ret, image = cam.read()
#cv2.imwrite('temp/temp.png', image)
#print(gethazard(image))
#print(getwavetypes(image, 3))
getweap(image, 4)
#getevent(image, 3)




#image = cv2.imread(path)
#cv2.imshow('Image', image)
#cv2.setMouseCallback('Image', click_event)
#cv2.startWindowThread()
#cv2.waitKey(0)
#cv2.destroyAllWindows()
#for i in range(2):
#    cv2.waitKey(1)

# Main process

#letters_detected = get_letters_from_image(path, threshold_binary, value_on_binary, value_off_binary, show=True, full_image=True, binary=binary)
#for i in range(len(letters_detected)):
    #cv2.imwrite('temp/'+str(i)+'.png', letters_detected[i])
#if len(letters_detected) != 16:
    #print("The number of detected letters is not 19. Please try again")
#plt.imshow(process_image(path, threshold_binary, value_on_binary, value_off_binary), cmap='gray')
#plt.imshow(cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2GRAY), cmap='gray')
#predicted_word = predict_word_from_letters(letters_detected, CHARS, show=False, binary=binary)
#print(predicted_word)


['GCharger, Glooga, Stamper, Charger',
 'Glooga, Naut, Stringer, Squelchies',
 'Jet, GCharger, GCharger, Hydra',
 'Jr, GCharger, Zap, Squeezer']